# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [1]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [2]:
pip install gymnasium --no-cache-dir -i https://pypi.org/simple/

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [4]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


/home/trueuser/.local/lib/python3.7/site-packages/gymnasium/envs/registration.py:524: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {env_name} is out of date. You should consider "


In [5]:
!python3 --version

Python 3.7.5


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [6]:
states = set(range(3))
actions = set(range(1))

print('states=' + str(states) + ' | actions=' + str(actions))

T = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}

print('Transition function T=' + str(T))

A_mdp = states, actions, T

states={0, 1, 2} | actions={0}
Transition function T={(0, 0): 1, (1, 0): 2, (2, 0): 2}


Попробуйте записать функцию перехода в матричном виде:

In [7]:
def transition_function_to_matrix(T):
    max_state = 0
    for key, next_state in T.items():
        s = key[0]
        if s > max_state:
            max_state = s
        if next_state > max_state:
            max_state = next_state
    
    n_states = max_state + 1

    counts = []
    for i in range(n_states):
        row = []
        for j in range(n_states):
            row.append(0.0)
        counts.append(row)

    actions_per_state = []
    for i in range(n_states):
        actions_per_state.append(0)

    for key, next_state in T.items():
        s = key[0]
        counts[s][next_state] += 1
        actions_per_state[s] += 1

    P = []
    for i in range(n_states):
        row = []
        for j in range(n_states):
            row.append(0.0)
        P.append(row)
    for s in range(n_states):
        if actions_per_state[s] > 0:
            for s_next in range(n_states):
                P[s][s_next] = counts[s][s_next] / actions_per_state[s]
        else:
            P[s][s] = 1.0
    P = np.array(P)

    return P

In [8]:
P = transition_function_to_matrix(T)
P

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [9]:
#A
P_start = [1, 0, 0]

def P_n(state, steps, matrix, start):
    ans = np.dot(start, np.linalg.matrix_power(matrix, steps))
    
    return str('Кол-во шагов: ' + str(steps) + '\n' + 'Вероятность: ' + str(ans) + '\n')
    
for n in [1, 100, 1000, 5000, 50000, 100000]:
    print(P_n(1, n, P, P_start))

Кол-во шагов: 1
Вероятность: [0. 1. 0.]

Кол-во шагов: 100
Вероятность: [0. 0. 1.]

Кол-во шагов: 1000
Вероятность: [0. 0. 1.]

Кол-во шагов: 5000
Вероятность: [0. 0. 1.]

Кол-во шагов: 50000
Вероятность: [0. 0. 1.]

Кол-во шагов: 100000
Вероятность: [0. 0. 1.]



In [10]:
#B

T_B = {} 
T_B[(0, 0)] = 1  
T_B[(0, 1)] = 2 
T_B[(0, 2)] = 3 
T_B[(1, 0)] = 1
T_B[(1, 1)] = 1
T_B[(1, 2)] = 1
T_B[(2, 0)] = 2
T_B[(2, 1)] = 2
T_B[(2, 2)] = 2
T_B[(3, 0)] = 3
T_B[(3, 1)] = 3
T_B[(3, 2)] = 3


states = set()
actions = set()

for i in range(4):
    states.add(i)
    
for i in range(3):
    actions.add(i)
B = states, actions, T_B

P_B = transition_function_to_matrix(T_B)

print(str(P_B) + '\n\n')

P_0_B = [1, 0, 0, 0]

steps_to_check = [1, 100, 1000, 5000, 50000, 100000]

for n in steps_to_check:
    P_n(1, n, P_B, P_0_B)

[[0.         0.33333333 0.33333333 0.33333333]
 [0.         1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]]




Задайте еще несколько MDP:

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

In [11]:
#C

T_C = {}
T_C[(0, 0)] = 1
T_C[(0, 1)] = 2
T_C[(1, 0)] = 1
T_C[(1, 1)] = 3
T_C[(2, 0)] = 3
T_C[(2, 1)] = 2
T_C[(3, 0)] = 3
T_C[(3, 1)] = 3

states = set()
actions = set()

for i in range(4):
    states.add(i)
for i in range(2):
    actions.add(i)

my_mdp_c = states, actions, T_C

trans_matrix_c = transition_function_to_matrix(T_C)

print(trans_matrix_c)

initial_p_c = [1, 0, 0, 0]

steps_to_check = [1, 100, 1000, 5000, 50000, 100000]

# Цикл: для каждого количества шагов
for n in steps_to_check:
    P_n(1, n, trans_matrix_c, initial_p_c)

[[0.  0.5 0.5 0. ]
 [0.  0.5 0.  0.5]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  1. ]]


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [12]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))

В качестве дополнения, запишите стратегию агента

In [13]:
class AgentA:
    def __init__(self, all_actions):
        self.act_list = list(all_actions)

    def choose_action(self, current_state):
        return 0

class AgentB:
    def __init__(self, all_actions):
        self.act_list = list(all_actions)
        self.n_act = len(self.act_list) 

    def choose_action(self, current_state):
        rand_num = np.random.randint(0, self.n_act)
        return rand_num

class AgentC:
    def __init__(self, all_actions):
        self.act_list = list(all_actions)
        self.n_act = len(self.act_list)

    def choose_action(self, current_state):
        action = current_state % 2
        return int(action)

## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [14]:
R = {
    (0, 0): -0.1,
    (1, 0): 1.0,
    (2, 0): 0.0
}
print(R)

A_mdp = *A_mdp, R
print(A_mdp)

{(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}
({0, 1, 2}, {0}, {(0, 0): 1, (1, 0): 2, (2, 0): 2}, {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0})


## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [15]:
R_B = {
    (0, 0): 0.05,
    (0, 1): 0.1,
    (0, 2): 0.15,
    (1, 0): 0.0,
    (1, 1): 0.0,
    (1, 2): 0.0,
    (2, 0): 0.0,
    (2, 1): 0.0,
    (2, 2): 0.0,
    (3, 0): 0.0,
    (3, 1): 0.0,
    (3, 2): 0.0
}


B = *B, R_B
print(B)
print()

R_C = {
    (0, 0): 0.05,
    (0, 1): 0.15,
    (1, 0): -0.05,
    (1, 1): 0.15,
    (2, 0): 0.1,
    (2, 1): -0.15,
    (3, 0): 0.0,
    (3, 1): 0.0
}

print(R_C)
my_mdp_c = *my_mdp_c, R_C
print(my_mdp_c)

({0, 1, 2, 3}, {0, 1, 2}, {(0, 0): 1, (0, 1): 2, (0, 2): 3, (1, 0): 1, (1, 1): 1, (1, 2): 1, (2, 0): 2, (2, 1): 2, (2, 2): 2, (3, 0): 3, (3, 1): 3, (3, 2): 3}, {(0, 0): 0.05, (0, 1): 0.1, (0, 2): 0.15, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0})

{(0, 0): 0.05, (0, 1): 0.15, (1, 0): -0.05, (1, 1): 0.15, (2, 0): 0.1, (2, 1): -0.15, (3, 0): 0.0, (3, 1): 0.0}
({0, 1, 2, 3}, {0, 1}, {(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 3, (2, 1): 2, (3, 0): 3, (3, 1): 3}, {(0, 0): 0.05, (0, 1): 0.15, (1, 0): -0.05, (1, 1): 0.15, (2, 0): 0.1, (2, 1): -0.15, (3, 0): 0.0, (3, 1): 0.0})


In [16]:
def run_episode(mdp):
    states, actions, T, R = mdp
    agent = Agent(actions)

    s = 0
    tau = []
    for _ in range(5):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

    return tau

run_episode(A_mdp)

[(0, 0, -0.1), (1, 0, 1.0), (2, 0, 0.0), (2, 0, 0.0), (2, 0, 0.0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.